# Advent of Code - Day 1: Report Repair


## Part One

After saving Christmas five years in a row, you've decided to take a vacation at
a nice resort on a tropical island. Surely, Christmas will go on without you.

The tropical island has its own currency and is entirely cash-only. The gold
coins used there have a little picture of a starfish; the locals just call them
stars. None of the currency exchanges seem to have heard of them, but somehow,
you'll need to find fifty of these coins by the time you arrive so you can pay
the deposit on your room.

To save your vacation, you need to get all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day
in the Advent calendar; the second puzzle is unlocked when you complete the
first. Each puzzle grants one star. Good luck!

Before you leave, the Elves in accounting just need you to fix your expense
report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find the two entries that sum to 2020 and then
multiply those two numbers together.

For example, suppose your expense report contained the following:

- 1721
- 979
- 366
- 299
- 675
- 1456

In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying
them together produces `1721 * 299 = 514579`, so the correct answer is 514579.

Of course, your expense report is much larger. Find the two entries that sum to
2020; what do you get if you multiply them together?


### Load Input data


:::{note} The input data can be found
[here](https://adventofcode.com/2020/day/1). :::


In [1]:
with open("../../data/advent-of-code/2020/day-1-input") as fid:
    data = fid.readlines()
    data = sorted([int(x.strip()) for x in data if x != "\n"])

In [2]:
data[:4], len(data)

([61, 156, 166, 279], 200)

### Brute force solution

My naive approach to this problem is to use two nested loops.


In [3]:
def pairs_addup(data, target):
    """Find indexes of pairs from data that add to target
    Parameters
    ----------
    data : list
      Input data
    target : int

    Returns
    -------
    n1 : int
    n2 : int
    """
    n1, n2 = None, None
    size = len(data)
    for x in range(size):
        for y in range(x + 1, size):
            if data[x] + data[y] == target:
                n1, n2 = data[x], data[y]
                break
    return n1, n2

In [4]:
a = %timeit -qo pairs_addup(data, 2020)
a

<TimeitResult : 2.07 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)>

In [5]:
n1, n2 = pairs_addup(data, 2020)
if n1 is not None:
    print(f"Two numbers: {n1} and {n2}\nTheir product: {n1 * n2}")
else:
    print("Found Nothing....")

Two numbers: 156 and 1864
Their product: 290784


- **This yields a solution with $O(N^2)$ time complexity in the worst case
  scenario**

Can we do better?


### Improved solution using `set` logic

Rather than using two nested for loops, a much better solution consists of using
**set logic** to find the two numbers that add up to the target:


In [6]:
def pairs_addup(data, target):
    rest = set()
    n1, n2 = None, None
    for num in data:
        remain = target - num
        if remain in rest:
            n1, n2 = num, remain
            break
        else:
            rest.add(remain)
    return n1, n2

In [7]:
b = %timeit -qo pairs_addup(data, 2020)
b

<TimeitResult : 21.9 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)>

In [8]:
pairs_addup(data, 2020)
n1, n2, n1 * n2

(156, 1864, 290784)

Compared to the brute force approach, this is a much faster approach with a
considerable speedup:


In [9]:
print(f"{(a.worst / b.worst):.0f}x Speedup")

99x Speedup


- **This yields a solution with $O(N * log(N))$ time complexity in the worst
  case scenario**. This includes the time complexity for sorting the list (as
  the list was sorted during the data loading step).


## Part Two

The Elves in accounting are thankful for your help; one of them even offers you
a starfish coin they had left over from a past vacation. They offer you a second
one if you can find three numbers in your expense report that meet the same
criteria.

Using the above example again, the three entries that sum to 2020 are 979, 366,
and 675. Multiplying them together produces the answer, 241861950.

In your expense report, what is the product of the three entries that sum to
2020?


### Brute force solution

A naive/brute force solution consists of using three nested for loops:


In [10]:
def triplets_addup(data, target):
    """Find indexes of triplets from data that add to target
    Parameters
    ----------
    data : list
      Input data
    target : int

    Returns
    -------
    n1 : int
    n2 : int
    n3 : int
    """
    n1, n2, n3 = None, None, None
    size = len(data)
    for x in range(size):
        for y in range(x + 1, size):
            for z in range(y + 1, size):
                if data[x] + data[y] + data[z] == target:
                    n1, n2, n3 = data[x], data[y], data[z]
                    break
    return n1, n2, n3

In [11]:
n1, n2, n3 = triplets_addup(data, 2020)
if n1 is not None:
    print(
        f"Three numbers: {n1} and {n2} and {n3}\nTheir product: {n1 * n2 * n3}"
    )
else:
    print("Found Nothing....")

Three numbers: 279 and 521 and 1220
Their product: 177337980


- **This yields a solution with 𝑂(𝑁^3) time complexity in the worst case
  scenario**


In [12]:
c = %timeit -qo triplets_addup(data, 2020)
c

<TimeitResult : 192 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

### Improved solution

Rather than using the nested three loops, we can save some work by:

- Looping over our input data,
- Saving the sums of all pairs of numbers in a dictionary with keys being the
  sum of each pair, and the values consisting of a tuple of numbers in each
  pair.
- Looping over our input data again
- Checking if `target - num` is in the dictionary


In [13]:
def triplets_addup(data, target):
    two_nums_sum = {}
    size = len(data)
    for x in range(size):
        for y in range(x + 1, size):
            two_nums_sum[data[x] + data[y]] = (data[x], data[y])

    n1, n2, n3 = None, None, None
    for num in data:
        remain = target - num
        if remain in two_nums_sum:
            n1, n2, n3 = num, *two_nums_sum[remain]
            break

    return n1, n2, n3

In [14]:
n1, n2, n3 = triplets_addup(data, 2020)
n1, n2, n3, n1 * n2 * n3

(279, 521, 1220, 177337980)

In [15]:
d = %timeit -qo triplets_addup(data, 2020)
d

<TimeitResult : 3.91 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)>

This solution provides a huge improvement over our brute force solution:


In [16]:
print(f"{(c.worst / d.worst):.0f}x Speedup")

47x Speedup


- **This yields a solution with 𝑂(𝑁^2) time complexity in the worst case
  scenario**
